In [ ]:
%pip install pandas
%pip install spacy
%pip install difflib
%pip install numpy
%pip install sklearn

In [ ]:
import pandas as pd
import spacy
from difflib import get_close_matches
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Descargar el modelo de lenguaje en español
!python -m spacy download es_core_news_md

# Cargar modelo de lenguaje
nlp = spacy.load("es_core_news_md")

# Cargar los archivos Excel
file_ceibal_combine = "../data/Ceibal.xlsx"
file_codeorg_combine = "../data/CodeOrg.xlsx"
file_thetech_combine = "../data/TheTech.xlsx"
file_restantes_combine = "../data/Restantes.xlsx"
file2 = "Enlaces.xlsx"

df_ceibal = pd.read_excel(file_ceibal_combine)
df_codeorg = pd.read_excel(file_codeorg_combine)
df_thetech = pd.read_excel(file_thetech_combine)
df_restantes = pd.read_excel(file_restantes_combine)

df1 = pd.concat([df_ceibal, df_codeorg, df_thetech, df_restantes], ignore_index=True)  # Datos principales combinados
df2 = pd.read_excel(file2, header=None, names=["Name", "URL"])  # Datos con URLs

In [ ]:
# Pasarlos a string
df1 = df1.astype(str)
df2 = df2.astype(str)

# Comprobar el tipo de nuevo
print(df1.dtypes)
print(df2.dtypes)

In [ ]:
# Crear una nueva columna URL en df1
df1["URL"] = ""

# Crear un diccionario basado en los nombres de df2
url_dict = {row["Name"].lower(): row["URL"] for _, row in df2.iterrows()}

def get_semantic_match(target, choices):
    """Encuentra la mejor coincidencia semántica usando embeddings de spaCy."""
    target_doc = nlp(target)
    best_match = None
    best_score = 0.0
    
    for choice in choices:
        choice_doc = nlp(choice)
        similarity = target_doc.similarity(choice_doc)
        if similarity > best_score:
            best_score = similarity
            best_match = choice
    
    return best_match if best_score > 0.7 else None  # Umbral de 0.7 para coincidencias confiables

# Intentar coincidencias exactas basadas en el Filename
for idx, row in df1.iterrows():
    filename = str(row["Filename"]).strip()
    name = str(row["Name"]).strip().lower()
    
    # Buscar coincidencia basada en filename
    for key, url in url_dict.items():
        if filename in url:
            df1.at[idx, "URL"] = url
            break
    
    # Si no encontró por filename, buscar por name (semántico)
    if not df1.at[idx, "URL"]:
        best_match = get_semantic_match(name, url_dict.keys())
        if best_match:
            df1.at[idx, "URL"] = url_dict[best_match]
        else:
            # Si no hay coincidencia semántica, buscar coincidencia basada en caracteres
            match = get_close_matches(name, url_dict.keys(), n=1, cutoff=0.6)
            if match:
                df1.at[idx, "URL"] = url_dict[match[0]]

# Guardar el resultado en un nuevo archivo Excel
output_file = "output.xlsx"
df1.to_excel(output_file, index=False)

print(f"Archivo guardado en {output_file}")

In [ ]:
# Mostrar el resultado
df1.head()